# Preprocess and external data notebook

## MLQA data

In [1]:
!wget https://dl.fbaipublicfiles.com/MLQA/MLQA_V1.zip

--2021-10-01 20:38:40--  https://dl.fbaipublicfiles.com/MLQA/MLQA_V1.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75719050 (72M) [application/zip]
Saving to: ‘MLQA_V1.zip’

MLQA_V1.zip         100%[===================>]  72.21M  9.33MB/s    in 8.7s    

2021-10-01 20:38:50 (8.28 MB/s) - ‘MLQA_V1.zip’ saved [75719050/75719050]



In [2]:
DATA_PATH = './data'

In [3]:
import zipfile
with zipfile.ZipFile('MLQA_V1.zip') as zip_ref:
    zip_ref.extractall(DATA_PATH)

In [4]:
import os
import sys
import random
import argparse
import json
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm

In [5]:
mlqa_train_data = f'{DATA_PATH}/MLQA_V1/dev/dev-context-hi-question-hi.json'
mlqa_test_data = f'{DATA_PATH}/MLQA_V1/test/test-context-hi-question-hi.json'
with open(mlqa_train_data, 'r') as file:
    train_data = json.load(file)
with open(mlqa_test_data, 'r') as file:
    test_data = json.load(file)

In [6]:
def preprocess(dataset, tier):
    num_exs = 0 
    examples = []
    for articles_id in tqdm(range(len(dataset['data'])), 
                            desc=f'preprocessing {tier}'):
        article_paragraphs = dataset['data'][articles_id]['paragraphs']
        for pid in range(len(article_paragraphs)):
            context = article_paragraphs[pid]['context']
            context = context.replace("''", '" ')
            context = context.replace("``", '" ')
            qas = article_paragraphs[pid]['qas'] 
            for qn in qas:
                question = qn['question'] 
                ans_text = qn['answers'][0]['text']
                ans_start_charloc = qn['answers'][0]['answer_start']
                ans_end_charloc = ans_start_charloc + len(ans_text)
                examples.append({
                    'context':context, 
                    'question':question, 
                    'answer_text':ans_text, 
                    'answer_start':ans_start_charloc
                })
                num_exs += 1
    print('num examples:', num_exs)
    return examples

In [7]:
examples_train = preprocess(train_data, 'dev')
examples_test = preprocess(test_data, 'test')

num examples: 507
num examples: 4918


preprocessing test: 100%|██████████| 2038/2038 [00:00<00:00, 160242.79it/s]


In [8]:
examples = examples_train + examples_test
mlqa = pd.DataFrame(examples)
mlqa['language'] = 'hindi'

## XQUAD data

In [9]:
!cd data/ && git clone https://github.com/deepmind/xquad.git

Cloning into 'xquad'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 38 (delta 12), reused 10 (delta 3), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [10]:
xquad_train_file = f'{DATA_PATH}/xquad/xquad.hi.json'
with open(xquad_train_file, 'r') as file:
    train_data = json.load(file)
examples_train = preprocess(train_data, 'dev')
xquad = pd.DataFrame(examples_train)
xquad['language'] = 'hindi'

num examples: 1190


preprocessing dev: 100%|██████████| 48/48 [00:00<00:00, 35576.35it/s]


## Save data to files

In [11]:
mlqa.to_csv(f'{DATA_PATH}/mlqa_hindi.csv', index=False)
xquad.to_csv(f'{DATA_PATH}/xquad.csv', index=False)

In [12]:
display(xquad.head())
display(mlqa.head())

,context,question,answer_text,answer_start,language
0,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,पैंथर्स डिफ़ेंस ने कितने अंक दिए?,308,35,hindi
1,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जेरेड एलन के पास कितने करियर सैक थे?,136,380,hindi
2,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,ल्यूक कुएक्ली ने कितने टैकल रजिस्टर किए?,118,743,hindi
3,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जोश नॉर्मन ने कितने बॉल को इंटरसेप्ट किया?,चार,90,hindi
4,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,इस सीज़न में टीम से किसने सबसे अधिक सैक रजिस्टर...,कावन शॉर्ट,169,hindi


,context,question,answer_text,answer_start,language
0,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,एक प्रकार की रस्सी का नाम बताईए जो सेलेरी के क...,कोलेन्काइमा,2267,hindi
1,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,क्या चीज पूरी तरह से पैरेनकाइमा कोशिकाओं से बन...,पत्तियां,158,hindi
2,जाइलेम कोशिकाएं कोशिका भित्तियों की लिग्निकृत ...,जाइलेम किस प्रकार के पौधों में पाया जाता है?,ट्रेकियोफाइटों,292,hindi
3,फ्लोएम उच्चतर पौधों में आहार का संवहन करने वाल...,लेप्टोम क्या है?\n,समान कार्य करने वाला एक अधिक सरल ऊतक,969,hindi
4,"सभी हवाई अवयवों की बाह्यत्वचा, जड़ों को छोड़कर...",बाह्यत्वचा की कोशिकाएं क्या कर सकती हैं?,क्यटिन का संश्लेषण,433,hindi
